In [78]:
import os
import sys
module_path = os.path.abspath(os.path.join('..\..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import binascii
import math
import numpy as np
from collections import deque
import importlib
print(module_path)

from libraries import encryption_standards as es
from libraries import crypto
from libraries import numeral

importlib.reload(es)
importlib.reload(crypto)
importlib.reload(numeral)

C:\Users\Oguz\Desktop\Cryptopals Challange


<module 'libraries.numeral' from 'C:\\Users\\Oguz\\Desktop\\Cryptopals Challange\\libraries\\numeral.py'>

<p style="color:royalblue; font-size:18px;">
    <b>
        Loading the Challange 7 Data
    </b>
</p>

In [2]:
data_path = "C:\\Users\\Oguz\\Desktop\\Cryptopals Challange\\challanges\\set_1\\challenge_7.txt"
data = np.loadtxt(data_path, dtype='str')

data_str = ""

for row in data:
    data_str += row
    
total_bytes = int((len(data_str) * 6) / 8)
number_of_blocks = int(total_bytes/16)

print("Text is ", total_bytes, "bytes long")
print("Which makes the total number of blocks ", number_of_blocks)

# data_str

Text is  2880 bytes long
Which makes the total number of blocks  180


<p style="color:royalblue; font-size:18px;">
    <b>
        Converting the CIPHERTEXT into column-major 4x4 16 byte blocks
    </b>
</p>

In [3]:
importlib.reload(crypto)

ch_7_obj = es.Encrypto(ciphertext=data_str, base=64)
ch_7_obj.Base16()

ch_7_key = crypto.Crypto(plaintext="YELLOW SUBMARINE")
print(ch_7_key.value)

blocksize = 128
block_bytes = 32

print(number_of_blocks)
# Divide the ciphertext into 16 byte blocks (each element is 2 hex digits = 8 bits = 1 byte)
blocks = [crypto.Crypto(ciphertext=ch_7_obj.value[i*block_bytes:(i+1)*block_bytes], base=16) for i in range(number_of_blocks)]

# Create the 16-byte states in column-major(Transpose)
states = [np.array([block.value[i*2:(i+1)*2] for i in range(16)]).reshape((4,4)).T for block in blocks]

print(states[0])

59454c4c4f57205355424d4152494e45
180
[['09' 'de' 'db' 'f8']
 ['12' '3e' 'aa' '8d']
 ['30' 'b3' '43' '2a']
 ['aa' '30' '58' '6c']]


<p style="color:royalblue; font-size:18px;">
    <b>
        Converting the KEY into column-major 4x4 16 byte blocks
    </b>
</p>

In [4]:
# Convert the key into a column-major 16 byte block
key_block = np.array([ch_7_key.value[i*2:(i+1)*2] for i in range(16)]).reshape((4,4)).T

print(key_block)

# Take the last column
# last_column = ""
"".join(key_block[:,-1])

[['59' '4f' '55' '52']
 ['45' '57' '42' '49']
 ['4c' '20' '4d' '4e']
 ['4c' '53' '41' '45']]


'52494e45'

<p style="color:royalblue; font-size:18px;">
    <b>
        Creating the RCON list
    </b>
</p>

In [6]:
deneme_block = ["a", "b", "c", "d"]

asd = deque(deneme_block)
print(asd)
asd.rotate(-1)
print(asd)

rc_list = ["01", "02", "04", "08", "10", "20", "40", "80", "1B", "36"]
rcon_list = [[rc] + ["00", "00", "00"] for rc in rc_list]

rcon_list

deque(['a', 'b', 'c', 'd'])
deque(['b', 'c', 'd', 'a'])


[['01', '00', '00', '00'],
 ['02', '00', '00', '00'],
 ['04', '00', '00', '00'],
 ['08', '00', '00', '00'],
 ['10', '00', '00', '00'],
 ['20', '00', '00', '00'],
 ['40', '00', '00', '00'],
 ['80', '00', '00', '00'],
 ['1B', '00', '00', '00'],
 ['36', '00', '00', '00']]

<p style="color:royalblue; font-size:18px;">
    <b>
        Creating the AES S-Box function
    </b>
</p>

In [76]:
# Load the AES S-Box data
s_box_path = "C:\\Users\\Oguz\\Desktop\\Cryptopals Challange\\challanges\\set_1\\s_box.txt"
s_box      = np.loadtxt(s_box_path, dtype='str')

# Delete the row and column and reshape into 1-D array
s_box = s_box[1:,1:].reshape(256)

# Create an array of hex values for 0 to 255
dec_objs = [numeral.Numeral(value=str(i), base=10) for i in range(256)]
[dec_obj.Base16() for dec_obj in dec_objs]
hex_list = [elem.value for elem in dec_objs]

# Now combine the to lists into one dictionary
s_box_dict = {(hex_key if len(hex_key)==2 else '0'+hex_key):s_value for hex_key, s_value in zip(hex_list, s_box)}

# s_box_dict

In [81]:
importlib.reload(es)

es.Encrypto.AES_S_Box("2f")

'15'